# Image to ASCII Characters

In [6]:
import cv2
import numpy as np

ASCII_CHARS = "@%#*+=:-. "

def image_to_ascii(image_path, output_txt_path, new_width=100):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    height, width = img.shape
    aspect_ratio = height / width
    new_height = int(aspect_ratio * new_width * 0.55)
    resized_img = cv2.resize(img, (new_width, new_height))

    ascii_img = ""
    for row in resized_img:
        for pixel in row:
            index = int(pixel / 256 * len(ASCII_CHARS))
            ascii_img += ASCII_CHARS[index]
        ascii_img += "\n"

    with open(output_txt_path, 'w') as f:
        f.write(ascii_img)

    print(f"ASCII art saved to: {output_txt_path}")

image_to_ascii("input.jpg", "ASCII_output.txt", new_width=120)


ASCII art saved to: ASCII_output.txt


# Image to Braille Art

In [7]:
import cv2
import numpy as np

def pixel_to_braille(block):
    # Mapping pixels to Braille dots (2x4 grid)
    dot_map = [
        (0, 0),  # Dot 1
        (1, 0),  # Dot 2
        (2, 0),  # Dot 3
        (0, 1),  # Dot 4
        (1, 1),  # Dot 5
        (2, 1),  # Dot 6
        (3, 0),  # Dot 7
        (3, 1)   # Dot 8
    ]

    braille_char = 0
    for i, (r, c) in enumerate(dot_map):
        if r < block.shape[0] and c < block.shape[1]:
            if block[r, c] == 255:  # White pixel = dot ON (image is inverted)
                braille_char |= (1 << i)

    return chr(0x2800 + braille_char)

def image_to_braille(image_path, output_txt_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("Error: Could not read image.")
        return

    # Resize image to multiple of 2 (width) and 4 (height)
    h, w = img.shape
    new_h = h
    new_w = w
    img = cv2.resize(img, (new_w, new_h))

    # Apply adaptive thresholding (inverted)
    img = cv2.adaptiveThreshold(
        img,
        maxValue=255,
        adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        thresholdType=cv2.THRESH_BINARY_INV,
        blockSize=11,
        C=2
    )

    braille_lines = []
    for y in range(0, new_h, 4):
        line = ''
        for x in range(0, new_w, 2):
            block = img[y:y+4, x:x+2]
            line += pixel_to_braille(block)
        braille_lines.append(line)

    # Write Braille output to text file
    with open(output_txt_path, 'w', encoding='utf-8') as f:
        for line in braille_lines:
            f.write(line + '\n')

    print(f"✅ Braille art saved to: {output_txt_path}")

# Example usage
if __name__ == "__main__":
    image_to_braille("input.jpg", "braille_art_output.txt")


✅ Braille art saved to: braille_art_output.txt
